In [3]:
pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Merging tokenizer training texts into a single file per language. 

import os

os.makedirs('output', exist_ok=True)

# Open output file in write mode
with open('output/merged_texts.txt', 'w', encoding='utf-8') as outfile:
    # Walk through the texts directory
    for root, dirs, files in os.walk('/Users/abu/Downloads/CustomLlamaTok/Yakut_train'):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                # Read each text file and write to merged file
                with open(file_path, 'r', encoding='utf-8') as infile:
                    outfile.write(infile.read())
                    outfile.write('\n\n')  # Add spacing between files

print("All text files have been merged into output/merged_texts.txt")

In [3]:
'''from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders
from transformers import PreTrainedTokenizerFast
import os

# 1. Gather multilingual corpus
corpus_files = [
    '/Users/abu/Downloads/CustomLlamaTok/Yakut_Train/SH_merged_texts.txt',         # Yakut
    '/Users/abu/Downloads/CustomLlamaTok/Russian_Train/RU_merged_texts.txt',                  # Russian
    '/Users/abu/Downloads/CustomLlamaTok/English_Train/EN_merged_texts.txt'                   # English
]

# 2. Create tokenizer with BPE model
tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))

# 3. Multilingual pre-tokenization
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.ByteLevel(add_prefix_space=True),
    pre_tokenizers.Punctuation(),
    pre_tokenizers.Digits(individual_digits=False)
])

# 4. Special tokens
special_tokens = [
    "<|begin_of_text|>", 
    "<|end_of_text|>", 
    "<unk>", 
    "<pad>",
    "<en>",  # English language token
    "<ru>",  # Russian language token
    "<sah>"  # Yakut language token
]

# 5. Trainer configuration for multilingual
trainer = trainers.BpeTrainer(
    vocab_size=50000,  # Larger vocabulary for multiple languages
    special_tokens=special_tokens,
    min_frequency=2,
    show_progress=True,
    limit_alphabet=2000,  # Larger alphabet for Cyrillic + Latin
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)

# 6. Train on multilingual data
print("Training multilingual tokenizer...")
tokenizer.train(files=corpus_files, trainer=trainer)
print("Training complete!")

# 7. Configure processing
tokenizer.decoder = decoders.ByteLevel()

# 8. Save tokenizer
tokenizer.save("multilingual_tokenizer.json")

# 9. Load as HF tokenizer
hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="multilingual_tokenizer.json",
    bos_token="<|begin_of_text|>",
    eos_token="<|end_of_text|>",
    unk_token="<unk>",
    pad_token="<pad>",
    additional_special_tokens=["<en>", "<ru>", "<sah>"]
)

# 10. Test all languages
def print_tokenization(text):
    tokens = hf_tokenizer.tokenize(text)
    decoded = [hf_tokenizer.convert_tokens_to_string([t]) for t in tokens]
    print(f"\nInput: {text}")
    print(f"Tokens: {tokens}")
    print(f"Decoded: {decoded}")

print("\nTesting tokenization:")
print_tokenization("<|begin_of_text|><sah>Мин аатым Кэскил.")  # Yakut
print_tokenization("<|begin_of_text|><ru>Привет, как дела?")   # Russian
print_tokenization("<|begin_of_text|><en>To fix this issue")    # English

# 11. Save tokenizer
save_path = "multilingual-tokenizer"
os.makedirs(save_path, exist_ok=True)
hf_tokenizer.save_pretrained(save_path)
print(f"\nTokenizer saved to: {save_path}")'''

Training multilingual tokenizer...



Training complete!

Testing tokenization:

Input: <|begin_of_text|><sah>Мин аатым Кэскил.
Tokens: ['<|begin_of_text|>', '<sah>', 'ĠÐľÐ¸Ð', '½', 'ĠÐ°Ð°ÑĤÑĭÐ', '¼', 'ĠÐļÑįÑģÐºÐ¸Ð', '»', '.']
Decoded: ['<|begin_of_text|>', '<sah>', ' Ми�', '�', ' ааты�', '�', ' Кэски�', '�', '.']

Input: <|begin_of_text|><ru>Привет, как дела?
Tokens: ['<|begin_of_text|>', '<ru>', 'ĠÐŁÑĢÐ¸Ð', '²', 'ÐµÑĤ', ',', 'ĠÐºÐ°Ðº', 'ĠÐ´ÐµÐ', '»', 'Ð°', '?']
Decoded: ['<|begin_of_text|>', '<ru>', ' При�', '�', 'ет', ',', ' как', ' де�', '�', 'а', '?']

Input: <|begin_of_text|><en>To fix this issue
Tokens: ['<|begin_of_text|>', '<en>', 'ĠTo', 'Ġfix', 'Ġthis', 'Ġissue']
Decoded: ['<|begin_of_text|>', '<en>', ' To', ' fix', ' this', ' issue']

Tokenizer saved to: multilingual-tokenizer


In [ ]:
#Trail 1; Multilingual Tokenizer; Suboptimal tokenization. 


from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders
from transformers import PreTrainedTokenizerFast
import os

# 1. Gather multilingual corpus
corpus_files = [
    '/Users/abu/Downloads/CustomLlamaTok/Yakut_Train/SH_merged_texts.txt',  # Yakut
    '/Users/abu/Downloads/CustomLlamaTok/Russian_Train/RU_merged_texts.txt',  # Russian
    '/Users/abu/Downloads/CustomLlamaTok/English_Train/EN_merged_texts.txt'   # English
]

# 2. Create tokenizer with BPE model
tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))

# 3. Improved multilingual pre-tokenization
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.UnicodeScripts(),  # Better for multilingual
    pre_tokenizers.Punctuation(),
    pre_tokenizers.Digits(individual_digits=False)
])

# 4. Special tokens
special_tokens = [
    "<|begin_of_text|>", 
    "<|end_of_text|>", 
    "<unk>", 
    "<pad>",
    "<en>",  # English language token
    "<ru>",  # Russian language token
    "<sah>"  # Yakut language token
]

# 5. Enhanced trainer configuration
trainer = trainers.BpeTrainer(
    vocab_size=50000,
    special_tokens=special_tokens,
    min_frequency=3,  # Increased frequency for better subwords
    show_progress=True,
    limit_alphabet=5000,  # Increased for Cyrillic + Latin
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet(),
    continuing_subword_prefix="▁",  # Add prefix for better word boundaries
)

# 6. Train on multilingual data
print("Training improved multilingual tokenizer...")
tokenizer.train(files=corpus_files, trainer=trainer)
print("Training complete!")

# 7. Configure processing
tokenizer.decoder = decoders.Sequence([
    decoders.Replace("▁", " "),  # Convert back to spaces
    decoders.ByteLevel()
])

# 8. Save tokenizer
tokenizer.save("yakut_tokenizer.json")

# 9. Load as HF tokenizer
hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="yakut_tokenizer.json",
    bos_token="<|begin_of_text|>",
    eos_token="<|end_of_text|>",
    unk_token="<unk>",
    pad_token="<pad>",
    additional_special_tokens=["<en>", "<ru>", "<sah>"]
)

# 10. Test all languages
def print_tokenization(text):
    tokens = hf_tokenizer.tokenize(text)
    decoded = hf_tokenizer.convert_tokens_to_string(tokens)
    print(f"\nInput: {text}")
    print(f"Tokens: {tokens}")
    print(f"Decoded: {decoded}")

print("\nTesting tokenization:")
print_tokenization("<|begin_of_text|><sah>Мин аатым Кэскил.")  # Yakut
print_tokenization("<|begin_of_text|><ru>Привет, как дела?")   # Russian
print_tokenization("<|begin_of_text|><en>To fix this issue")    # English

# 11. Save tokenizer
save_path = "yakut_tokenizer"
os.makedirs(save_path, exist_ok=True)
hf_tokenizer.save_pretrained(save_path)
print(f"\nTokenizer saved to: {save_path}")

Training improved multilingual tokenizer...



In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Load your Yakut tokenizer
yakut_tokenizer = AutoTokenizer.from_pretrained("yakut-tokenizer")

# 2. Load Llama 3 model
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3-1B",
    token="hf_iwTFtLrOyizDmnYjwmlGCLwGMhOADK"
)

# 3. Resize embeddings
original_vocab_size = model.get_input_embeddings().weight.size(0)
new_vocab_size = len(yakut_tokenizer)
model.resize_token_embeddings(new_vocab_size)

# 4. Initialize new embeddings
print(f"\nInitializing new embeddings...")
with torch.no_grad():
    # Copy embeddings for tokens that exist in both vocabularies
    for token, idx in yakut_tokenizer.get_vocab().items():
        if idx < original_vocab_size:
            # Only initialize new tokens
            continue
        # Initialize with mean of existing embeddings
        model.get_input_embeddings().weight[idx] = model.get_input_embeddings().weight.mean(dim=0)
    
    # Tie weights if needed
    if model.config.tie_word_embeddings:
        model.lm_head.weight = model.get_input_embeddings().weight

# 5. Save the custom model
model.save_pretrained("yakut-llama-model")
yakut_tokenizer.save_pretrained("yakut-llama-model")
print("Custom model saved!")

OSError: yakut-tokenizer is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Merging tokenizer training texts into a single file per language. 

import os

os.makedirs('output', exist_ok=True)

# Open output file in write mode
with open('output/merged_texts.txt', 'w', encoding='utf-8') as outfile:
    # Walk through the texts directory
    for root, dirs, files in os.walk('/Users/abu/Downloads/CustomLlamaTok/Yakut_train'):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                # Read each text file and write to merged file
                with open(file_path, 'r', encoding='utf-8') as infile:
                    outfile.write(infile.read())
                    outfile.write('\n\n')  # Add spacing between files

print("All text files have been merged into output/merged_texts.txt")

In [ ]:
'''from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders
from transformers import PreTrainedTokenizerFast
import os

# 1. Gather multilingual corpus
corpus_files = [
    '/Users/abu/Downloads/CustomLlamaTok/Yakut_Train/SH_merged_texts.txt',         # Yakut
    '/Users/abu/Downloads/CustomLlamaTok/Russian_Train/RU_merged_texts.txt',                  # Russian
    '/Users/abu/Downloads/CustomLlamaTok/English_Train/EN_merged_texts.txt'                   # English
]

# 2. Create tokenizer with BPE model
tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))

# 3. Multilingual pre-tokenization
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.ByteLevel(add_prefix_space=True),
    pre_tokenizers.Punctuation(),
    pre_tokenizers.Digits(individual_digits=False)
])

# 4. Special tokens
special_tokens = [
    "<|begin_of_text|>", 
    "<|end_of_text|>", 
    "<unk>", 
    "<pad>",
    "<en>",  # English language token
    "<ru>",  # Russian language token
    "<sah>"  # Yakut language token
]

# 5. Trainer configuration for multilingual
trainer = trainers.BpeTrainer(
    vocab_size=50000,  # Larger vocabulary for multiple languages
    special_tokens=special_tokens,
    min_frequency=2,
    show_progress=True,
    limit_alphabet=2000,  # Larger alphabet for Cyrillic + Latin
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)

# 6. Train on multilingual data
print("Training multilingual tokenizer...")
tokenizer.train(files=corpus_files, trainer=trainer)
print("Training complete!")

# 7. Configure processing
tokenizer.decoder = decoders.ByteLevel()

# 8. Save tokenizer
tokenizer.save("multilingual_tokenizer.json")

# 9. Load as HF tokenizer
hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="multilingual_tokenizer.json",
    bos_token="<|begin_of_text|>",
    eos_token="<|end_of_text|>",
    unk_token="<unk>",
    pad_token="<pad>",
    additional_special_tokens=["<en>", "<ru>", "<sah>"]
)

# 10. Test all languages
def print_tokenization(text):
    tokens = hf_tokenizer.tokenize(text)
    decoded = [hf_tokenizer.convert_tokens_to_string([t]) for t in tokens]
    print(f"\nInput: {text}")
    print(f"Tokens: {tokens}")
    print(f"Decoded: {decoded}")

print("\nTesting tokenization:")
print_tokenization("<|begin_of_text|><sah>Мин аатым Кэскил.")  # Yakut
print_tokenization("<|begin_of_text|><ru>Привет, как дела?")   # Russian
print_tokenization("<|begin_of_text|><en>To fix this issue")    # English

# 11. Save tokenizer
save_path = "multilingual-tokenizer"
os.makedirs(save_path, exist_ok=True)
hf_tokenizer.save_pretrained(save_path)
print(f"\nTokenizer saved to: {save_path}")'''

Training multilingual tokenizer...



Training complete!

Testing tokenization:

Input: <|begin_of_text|><sah>Мин аатым Кэскил.
Tokens: ['<|begin_of_text|>', '<sah>', 'ĠÐľÐ¸Ð', '½', 'ĠÐ°Ð°ÑĤÑĭÐ', '¼', 'ĠÐļÑįÑģÐºÐ¸Ð', '»', '.']
Decoded: ['<|begin_of_text|>', '<sah>', ' Ми�', '�', ' ааты�', '�', ' Кэски�', '�', '.']

Input: <|begin_of_text|><ru>Привет, как дела?
Tokens: ['<|begin_of_text|>', '<ru>', 'ĠÐŁÑĢÐ¸Ð', '²', 'ÐµÑĤ', ',', 'ĠÐºÐ°Ðº', 'ĠÐ´ÐµÐ', '»', 'Ð°', '?']
Decoded: ['<|begin_of_text|>', '<ru>', ' При�', '�', 'ет', ',', ' как', ' де�', '�', 'а', '?']

Input: <|begin_of_text|><en>To fix this issue
Tokens: ['<|begin_of_text|>', '<en>', 'ĠTo', 'Ġfix', 'Ġthis', 'Ġissue']
Decoded: ['<|begin_of_text|>', '<en>', ' To', ' fix', ' this', ' issue']

Tokenizer saved to: multilingual-tokenizer


In [ ]:
#Trail 1; Multilingual Tokenizer; Suboptimal tokenization. 


from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders
from transformers import PreTrainedTokenizerFast
import os

# 1. Gather multilingual corpus
corpus_files = [
    '/Users/abu/Downloads/CustomLlamaTok/Yakut_Train/SH_merged_texts.txt',  # Yakut
    '/Users/abu/Downloads/CustomLlamaTok/Russian_Train/RU_merged_texts.txt',  # Russian
    '/Users/abu/Downloads/CustomLlamaTok/English_Train/EN_merged_texts.txt'   # English
]

# 2. Create tokenizer with BPE model
tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))

# 3. Improved multilingual pre-tokenization
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.UnicodeScripts(),  # Better for multilingual
    pre_tokenizers.Punctuation(),
    pre_tokenizers.Digits(individual_digits=False)
])

# 4. Special tokens
special_tokens = [
    "<|begin_of_text|>", 
    "<|end_of_text|>", 
    "<unk>", 
    "<pad>",
    "<en>",  # English language token
    "<ru>",  # Russian language token
    "<sah>"  # Yakut language token
]

# 5. Enhanced trainer configuration
trainer = trainers.BpeTrainer(
    vocab_size=50000,
    special_tokens=special_tokens,
    min_frequency=3,  # Increased frequency for better subwords
    show_progress=True,
    limit_alphabet=5000,  # Increased for Cyrillic + Latin
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet(),
    continuing_subword_prefix="▁",  # Add prefix for better word boundaries
)

# 6. Train on multilingual data
print("Training improved multilingual tokenizer...")
tokenizer.train(files=corpus_files, trainer=trainer)
print("Training complete!")

# 7. Configure processing
tokenizer.decoder = decoders.Sequence([
    decoders.Replace("▁", " "),  # Convert back to spaces
    decoders.ByteLevel()
])

# 8. Save tokenizer
tokenizer.save("yakut_tokenizer.json")

# 9. Load as HF tokenizer
hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="yakut_tokenizer.json",
    bos_token="<|begin_of_text|>",
    eos_token="<|end_of_text|>",
    unk_token="<unk>",
    pad_token="<pad>",
    additional_special_tokens=["<en>", "<ru>", "<sah>"]
)

# 10. Test all languages
def print_tokenization(text):
    tokens = hf_tokenizer.tokenize(text)
    decoded = hf_tokenizer.convert_tokens_to_string(tokens)
    print(f"\nInput: {text}")
    print(f"Tokens: {tokens}")
    print(f"Decoded: {decoded}")

print("\nTesting tokenization:")
print_tokenization("<|begin_of_text|><sah>Мин аатым Кэскил.")  # Yakut
print_tokenization("<|begin_of_text|><ru>Привет, как дела?")   # Russian
print_tokenization("<|begin_of_text|><en>To fix this issue")    # English

# 11. Save tokenizer
save_path = "yakut_tokenizer"
os.makedirs(save_path, exist_ok=True)
hf_tokenizer.save_pretrained(save_path)
print(f"\nTokenizer saved to: {save_path}")

Training improved multilingual tokenizer...



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Load your Yakut tokenizer
yakut_tokenizer = AutoTokenizer.from_pretrained("yakut-tokenizer")

# 2. Load Llama 3 model
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3-1B",
    token="hf_iwTFtLrOyizDmnYjwmlGCLwGMhOADK"
)

# 3. Resize embeddings
original_vocab_size = model.get_input_embeddings().weight.size(0)
new_vocab_size = len(yakut_tokenizer)
model.resize_token_embeddings(new_vocab_size)

# 4. Initialize new embeddings
print(f"\nInitializing new embeddings...")
with torch.no_grad():
    # Copy embeddings for tokens that exist in both vocabularies
    for token, idx in yakut_tokenizer.get_vocab().items():
        if idx < original_vocab_size:
            # Only initialize new tokens
            continue
        # Initialize with mean of existing embeddings
        model.get_input_embeddings().weight[idx] = model.get_input_embeddings().weight.mean(dim=0)
    
    # Tie weights if needed
    if model.config.tie_word_embeddings:
        model.lm_head.weight = model.get_input_embeddings().weight

# 5. Save the custom model
model.save_pretrained("yakut-llama-model")
yakut_tokenizer.save_pretrained("yakut-llama-model")
print("Custom model saved!")

OSError: yakut-tokenizer is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
from huggingface_hub import login

# Login to Hugging Face
login(token="hf_iwTFtLrOyizDmnYjwmlGCLwGMhOADK")


In [ ]:
#Trail 2: Multilingual Tokenizer; decent tokenization. 


from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Load your Yakut tokenizer
tokenizer_path = "/Users/abu/Downloads/CustomLlamaTok/yakut_tokenizer"
yakut_tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# 2. Load Llama 3 model and tokenizer
model_name = "meta-llama/Llama-3.2-1B"
token = "hf_ynsefXAVByqhClmLjMARNwGQJkBikq"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=token
)

# Load the ORIGINAL tokenizer for Llama
original_tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=token
)

# 3. Get vocabulary sizes
original_vocab_size = model.config.vocab_size
new_vocab_size = len(yakut_tokenizer)
print(f"Original vocab size: {original_vocab_size}")
print(f"New vocab size: {new_vocab_size}")

# 4. Create new embedding matrix
print("\nCreating new embeddings...")
with torch.no_grad():
    # Get original embeddings
    original_embeddings = model.get_input_embeddings().weight.data.clone()
    mean_embedding = original_embeddings.mean(dim=0)
    
    # Create new embedding layer
    new_embeddings = torch.nn.Embedding(new_vocab_size, model.config.hidden_size)
    
    # Get vocabulary of the original tokenizer
    original_vocab = original_tokenizer.get_vocab()
    
    # Copy embeddings for overlapping tokens
    for token, new_idx in yakut_tokenizer.get_vocab().items():
        # Check if token exists in original tokenizer
        if token in original_vocab:
            old_idx = original_vocab[token]
            new_embeddings.weight[new_idx] = original_embeddings[old_idx].clone()
        else:
            # For new tokens, use mean embedding
            new_embeddings.weight[new_idx] = mean_embedding
    
    # Replace model embeddings
    model.set_input_embeddings(new_embeddings)
    
    # Update config
    model.config.vocab_size = new_vocab_size
    
    # Tie weights if needed
    if model.config.tie_word_embeddings:
        model.lm_head.weight = model.get_input_embeddings().weight

# 5. Save the custom model
save_path = "yakut-llama-model"
model.save_pretrained(save_path)
yakut_tokenizer.save_pretrained(save_path)
print(f"\nCustom model saved to: {save_path}")

# 6. Test the model with proper settings
test_text = "<|begin_of_text|>Мин аатым Кэскил."
inputs = yakut_tokenizer(test_text, return_tensors="pt")

# Set pad token if not set
if yakut_tokenizer.pad_token is None:
    yakut_tokenizer.pad_token = yakut_tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

print("\nTest inference:")
with torch.no_grad():
    # Forward pass
    outputs = model(**inputs)
    print(f"Logits shape: {outputs.logits.shape}")
    
    # Test generation with attention mask
    generated = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=20,
        do_sample=True,
        top_k=50,
        temperature=0.7,
        pad_token_id=yakut_tokenizer.pad_token_id
    )
    decoded = yakut_tokenizer.decode(generated[0], skip_special_tokens=False)
    print(f"Generated text: {decoded}")

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Original vocab size: 128256
New vocab size: 50000

Creating new embeddings...

Custom model saved to: yakut-llama-model

Test inference:
Logits shape: torch.Size([1, 6, 50000])
Generated text: <|begin_of_text|>Мин  аатым  К эскил.Indo,Indo,Indo,Indo,Indo,Indo,Indo


**RUSSIAN**


In [ ]:
# Tokenization Trail using trail 1 Multilingual Tokenizer; Suboptimal tokenization.     


from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


# Load your custom model and tokenizer
model = AutoModelForCausalLM.from_pretrained("yakut-llama-model")
tokenizer = AutoTokenizer.from_pretrained("yakut-llama-model")




# Test tokenization
text = "<|begin_of_text|>Жил-был царь. Добрый царь, который очень любил своих подданных и был с ними ласков."
inputs = tokenizer(text, return_tensors="pt")

print("Tokenization test:")
print(f"Input text: {text}")
print(f"Token IDs: {inputs['input_ids'].tolist()[0]}")
print(f"Tokens: {tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])}")

# Test forward pass
with torch.no_grad():
    outputs = model(**inputs)

print("\nForward pass test:")
print(f"Logits shape: {outputs.logits.shape}")  # Should be [1, seq_len, vocab_size]
print("Forward pass successful!")

# Test embedding layer
input_embeds = model.get_input_embeddings()(inputs.input_ids)
print(f"\nEmbedding test:")
print(f"Embedding shape: {input_embeds.shape}")  # Should be [1, seq_len, hidden_size]
print(f"Embedding norms: {torch.norm(input_embeds, dim=2).mean().item():.4f}")

Using device: cuda


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Tokenization test:
Input text: <|begin_of_text|>Жил-был царь. Добрый царь, который очень любил своих подданных и был с ними ласков.
Token IDs: [0, 2696, 1031, 17, 3091, 4415, 8687, 18, 1138, 20195, 1162, 4415, 8687, 16, 20548, 1162, 4498, 6162, 18406, 3685, 17454, 4297, 4450, 382, 1197, 10381, 1058, 1401, 1009, 1121, 20132, 1388, 25376, 18]
Tokens: ['<|begin_of_text|>', '▁Ж', 'ил', '-', 'был', '▁ц', 'арь', '.', '▁Д', 'обр', 'ый', '▁ц', 'арь', ',', '▁котор', 'ый', '▁оч', 'ень', '▁лю', 'бил', '▁сво', 'их', '▁под', 'д', 'анн', 'ых', '▁и', '▁был', '▁с', '▁н', 'ими', '▁л', 'асков', '.']

Forward pass test:
Logits shape: torch.Size([1, 34, 30000])
Forward pass successful!

Embedding test:
Embedding shape: torch.Size([1, 34, 4096])
Embedding norms: 0.5088


In [ ]:
# Tokenization Trail using trail 2 Multilingual Tokenizer; decent tokenization.     

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load your custom model and tokenizer
model = AutoModelForCausalLM.from_pretrained("yakut-llama-model")
tokenizer = AutoTokenizer.from_pretrained("yakut-llama-model")

# Test texts in different languages
test_texts = {
    "Yakut": "<|begin_of_text|>Араас сылларга үөрэнэн ааспыт училещаларын үбүлүөйүн көрсө кэллилэр ыраахтан-чугастан элбэх ыалдьыт",
    "Russian": "<|begin_of_text|>Жил-был царь. Добрый царь, который очень любил своих подданных и был с ними ласков.",
    "English": "<|begin_of_text|>Once upon a time there was a king. A kind king who loved his subjects dearly."
}

# Run tests for each language
for lang, text in test_texts.items():
    print(f"\n{'='*40}")
    print(f"Testing {lang} text")
    print(f"{'='*40}")
    
    # Tokenization test
    inputs = tokenizer(text, return_tensors="pt")
    
    print("\nTokenization:")
    print(f"Input text: {text}")
    print(f"Token IDs: {inputs['input_ids'].tolist()[0]}")
    print(f"Tokens: {tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])}")
    
    # Forward pass test
    with torch.no_grad():
        outputs = model(**inputs)
    
    print("\nForward pass:")
    print(f"Logits shape: {outputs.logits.shape}")  # Should be [1, seq_len, vocab_size]
    
    # Embedding test
    input_embeds = model.get_input_embeddings()(inputs.input_ids)
    print("\nEmbeddings:")
    print(f"Embedding shape: {input_embeds.shape}")
    print(f"Mean embedding norm: {torch.norm(input_embeds, dim=2).mean().item():.4f}")
    
    # Test generation
    print("\nTest generation:")
    with torch.no_grad():
        generated = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=20,
            do_sample=True,
            top_k=50,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id
        )
        decoded = tokenizer.decode(generated[0], skip_special_tokens=False)
        print(f"Generated text: {decoded}")
    
    print(f"\n{'='*40}\n")


Testing Yakut text

Tokenization:
Input text: <|begin_of_text|>Араас сылларга үөрэнэн ааспыт училещаларын үбүлүөйүн көрсө кэллилэр ыраахтан-чугастан элбэх ыалдьыт
Token IDs: [0, 7931, 5567, 16944, 16780, 17172, 5546, 4560, 49601, 8374, 38655, 12512, 7897, 17392, 5281, 14709, 24109, 5366, 21, 27231, 32396, 7412, 49479]
Tokens: ['<|begin_of_text|>', 'Ар', '▁аас', '▁ сылларга ', '▁үөрэнэн ', '▁ааспыт ', '▁уч', '▁ил', '▁ещ', '▁аларын ', '▁үбүлүөй', '▁үн к', '▁өрс', '▁ө к', '▁элл', '▁илэр ', '▁ыраах', '▁тан', '-', 'чуг', '▁астан ', '▁элбэх ', '▁ыалдьыт']

Forward pass:
Logits shape: torch.Size([1, 23, 50000])

Embeddings:
Embedding shape: torch.Size([1, 23, 2048])
Mean embedding norm: 0.4007

Test generation:
Generated text: <|begin_of_text|>Ар аас  сылларга  үөрэнэн  ааспыт  уч ил ещ аларын  үбүлүөй үн к өрс ө к элл илэр  ыраах тан-чуг астан  элбэх  ыалдьыт-Ɵ-Ɵ-Ɵ-Ɵ-Ɵ-Ɵ-�



Testing Russian text

Tokenization:
Input text: <|begin_of_text|>Жил-был царь. Добрый царь, который очень любил свои

In [ ]:
from transformers import AutoTokenizer
import os

# Load your custom tokenizer
tokenizer_path = "yakut-llama-model"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

def analyze_tokenization(text, tokenizer):
    # Encode the text
    inputs = tokenizer(text, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    # Calculate metrics
    char_count = len(text)
    token_count = len(tokens)
    compression_ratio = char_count / token_count
    
    # Reconstruct text from tokens
    reconstructed = tokenizer.decode(inputs['input_ids'][0])
    
    return {
        "original": text,
        "tokens": tokens,
        "token_count": token_count,
        "char_count": char_count,
        "compression_ratio": compression_ratio,
        "reconstructed": reconstructed,
        "perfect_reconstruction": text == reconstructed
    }

def print_tokenization_report(report):
    print(f"\nOriginal text ({report['char_count']} chars):")
    print(report['original'])
    
    print(f"\nTokens ({report['token_count']} tokens, ratio: {report['compression_ratio']:.2f} chars/token):")
    for i, token in enumerate(report['tokens']):
        print(f"{i:3d}: {token}")
    
    print("\nReconstructed text:")
    print(report['reconstructed'])
    print(f"Perfect reconstruction: {report['perfect_reconstruction']}")
    
    # Highlight problematic tokens
    problem_tokens = [token for token in report['tokens'] if len(token) == 1 or '�' in token]
    if problem_tokens:
        print("\nWarning: Problematic tokens detected:")
        print(problem_tokens)

# Test texts
test_texts = {
    "Yakut": "<|begin_of_text|>Араас сылларга үөрэнэн ааспыт училещаларын үбүлүөйүн көрсө кэллилэр ыраахтан-чугастан элбэх ыалдьыт",
    "Russian": "<|begin_of_text|>Жил-был царь. Добрый царь, который очень любил своих подданных и был с ними ласков.",
    "English": "<|begin_of_text|>Once upon a time there was a king. A kind king who loved his subjects dearly."
}

# Run analysis
for lang, text in test_texts.items():
    print(f"\n{'='*60}")
    print(f"TOKENIZATION ANALYSIS: {lang.upper()}")
    print(f"{'='*60}")
    
    report = analyze_tokenization(text, tokenizer)
    print_tokenization_report(report)


TOKENIZATION ANALYSIS: YAKUT

Original text (116 chars):
<|begin_of_text|>Араас сылларга үөрэнэн ааспыт училещаларын үбүлүөйүн көрсө кэллилэр ыраахтан-чугастан элбэх ыалдьыт

Tokens (23 tokens, ratio: 5.04 chars/token):
  0: <|begin_of_text|>
  1: Ар
  2: ▁аас
  3: ▁ сылларга 
  4: ▁үөрэнэн 
  5: ▁ааспыт 
  6: ▁уч
  7: ▁ил
  8: ▁ещ
  9: ▁аларын 
 10: ▁үбүлүөй
 11: ▁үн к
 12: ▁өрс
 13: ▁ө к
 14: ▁элл
 15: ▁илэр 
 16: ▁ыраах
 17: ▁тан
 18: -
 19: чуг
 20: ▁астан 
 21: ▁элбэх 
 22: ▁ыалдьыт

Reconstructed text:
<|begin_of_text|>Ар аас  сылларга  үөрэнэн  ааспыт  уч ил ещ аларын  үбүлүөй үн к өрс ө к элл илэр  ыраах тан-чуг астан  элбэх  ыалдьыт
Perfect reconstruction: False

['-']

TOKENIZATION ANALYSIS: RUSSIAN

Original text (100 chars):
<|begin_of_text|>Жил-был царь. Добрый царь, который очень любил своих подданных и был с ними ласков.

Tokens (40 tokens, ratio: 2.50 chars/token):
  0: <|begin_of_text|>
  1: Ж
  2: ▁ил
  3: -
  4: б
  5: ▁ыл 
  6: ▁ц
  7: ▁арь
  8: .
  9:  
 10: Д
 11

In [26]:
from huggingface_hub import login

# Login to Hugging Face
login(token="hf_iwTFtLrOyizDmnYjwmlGCLwGMhOADK")


In [ ]:
#Trail 2: Multilingual Tokenizer; decent tokenization. 


from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Load your Yakut tokenizer
tokenizer_path = "/Users/abu/Downloads/CustomLlamaTok/yakut_tokenizer"
yakut_tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# 2. Load Llama 3 model and tokenizer
model_name = "meta-llama/Llama-3.2-1B"
token = "hf_ynsefXAVByqhClmLjMARNwGQJkBikq"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=token
)

# Load the ORIGINAL tokenizer for Llama
original_tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=token
)

# 3. Get vocabulary sizes
original_vocab_size = model.config.vocab_size
new_vocab_size = len(yakut_tokenizer)
print(f"Original vocab size: {original_vocab_size}")
print(f"New vocab size: {new_vocab_size}")

# 4. Create new embedding matrix
print("\nCreating new embeddings...")
with torch.no_grad():
    # Get original embeddings
    original_embeddings = model.get_input_embeddings().weight.data.clone()
    mean_embedding = original_embeddings.mean(dim=0)
    
    # Create new embedding layer
    new_embeddings = torch.nn.Embedding(new_vocab_size, model.config.hidden_size)
    
    # Get vocabulary of the original tokenizer
    original_vocab = original_tokenizer.get_vocab()
    
    # Copy embeddings for overlapping tokens
    for token, new_idx in yakut_tokenizer.get_vocab().items():
        # Check if token exists in original tokenizer
        if token in original_vocab:
            old_idx = original_vocab[token]
            new_embeddings.weight[new_idx] = original_embeddings[old_idx].clone()
        else:
            # For new tokens, use mean embedding
            new_embeddings.weight[new_idx] = mean_embedding
    
    # Replace model embeddings
    model.set_input_embeddings(new_embeddings)
    
    # Update config
    model.config.vocab_size = new_vocab_size
    
    # Tie weights if needed
    if model.config.tie_word_embeddings:
        model.lm_head.weight = model.get_input_embeddings().weight

# 5. Save the custom model
save_path = "yakut-llama-model"
model.save_pretrained(save_path)
yakut_tokenizer.save_pretrained(save_path)
print(f"\nCustom model saved to: {save_path}")

# 6. Test the model with proper settings
test_text = "<|begin_of_text|>Мин аатым Кэскил."
inputs = yakut_tokenizer(test_text, return_tensors="pt")

# Set pad token if not set
if yakut_tokenizer.pad_token is None:
    yakut_tokenizer.pad_token = yakut_tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

print("\nTest inference:")
with torch.no_grad():
    # Forward pass
    outputs = model(**inputs)
    print(f"Logits shape: {outputs.logits.shape}")
    
    # Test generation with attention mask
    generated = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=20,
        do_sample=True,
        top_k=50,
        temperature=0.7,
        pad_token_id=yakut_tokenizer.pad_token_id
    )
    decoded = yakut_tokenizer.decode(generated[0], skip_special_tokens=False)
    print(f"Generated text: {decoded}")

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Original vocab size: 128256
New vocab size: 50000

Creating new embeddings...

Custom model saved to: yakut-llama-model

Test inference:
Logits shape: torch.Size([1, 6, 50000])
Generated text: <|begin_of_text|>Мин  аатым  К эскил.Indo,Indo,Indo,Indo,Indo,Indo,Indo


**RUSSIAN**


In [7]:
# Tokenization Trail using trail 1 Multilingual Tokenizer; Suboptimal tokenization.     


from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


# Load your custom model and tokenizer
model = AutoModelForCausalLM.from_pretrained("yakut-llama-model")
tokenizer = AutoTokenizer.from_pretrained("yakut-llama-model")




# Test tokenization
text = "<|begin_of_text|>Жил-был царь. Добрый царь, который очень любил своих подданных и был с ними ласков."
inputs = tokenizer(text, return_tensors="pt")

print("Tokenization test:")
print(f"Input text: {text}")
print(f"Token IDs: {inputs['input_ids'].tolist()[0]}")
print(f"Tokens: {tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])}")

# Test forward pass
with torch.no_grad():
    outputs = model(**inputs)

print("\nForward pass test:")
print(f"Logits shape: {outputs.logits.shape}")  # Should be [1, seq_len, vocab_size]
print("Forward pass successful!")

# Test embedding layer
input_embeds = model.get_input_embeddings()(inputs.input_ids)
print(f"\nEmbedding test:")
print(f"Embedding shape: {input_embeds.shape}")  # Should be [1, seq_len, hidden_size]
print(f"Embedding norms: {torch.norm(input_embeds, dim=2).mean().item():.4f}")

Using device: cuda


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Tokenization test:
Input text: <|begin_of_text|>Жил-был царь. Добрый царь, который очень любил своих подданных и был с ними ласков.
Token IDs: [0, 2696, 1031, 17, 3091, 4415, 8687, 18, 1138, 20195, 1162, 4415, 8687, 16, 20548, 1162, 4498, 6162, 18406, 3685, 17454, 4297, 4450, 382, 1197, 10381, 1058, 1401, 1009, 1121, 20132, 1388, 25376, 18]
Tokens: ['<|begin_of_text|>', '▁Ж', 'ил', '-', 'был', '▁ц', 'арь', '.', '▁Д', 'обр', 'ый', '▁ц', 'арь', ',', '▁котор', 'ый', '▁оч', 'ень', '▁лю', 'бил', '▁сво', 'их', '▁под', 'д', 'анн', 'ых', '▁и', '▁был', '▁с', '▁н', 'ими', '▁л', 'асков', '.']

Forward pass test:
Logits shape: torch.Size([1, 34, 30000])
Forward pass successful!

Embedding test:
Embedding shape: torch.Size([1, 34, 4096])
Embedding norms: 0.5088


In [38]:
# Tokenization Trail using trail 2 Multilingual Tokenizer; decent tokenization.     

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load your custom model and tokenizer
model = AutoModelForCausalLM.from_pretrained("yakut-llama-model")
tokenizer = AutoTokenizer.from_pretrained("yakut-llama-model")

# Test texts in different languages
test_texts = {
    "Yakut": "<|begin_of_text|>Араас сылларга үөрэнэн ааспыт училещаларын үбүлүөйүн көрсө кэллилэр ыраахтан-чугастан элбэх ыалдьыт",
    "Russian": "<|begin_of_text|>Жил-был царь. Добрый царь, который очень любил своих подданных и был с ними ласков.",
    "English": "<|begin_of_text|>Once upon a time there was a king. A kind king who loved his subjects dearly."
}

# Run tests for each language
for lang, text in test_texts.items():
    print(f"\n{'='*40}")
    print(f"Testing {lang} text")
    print(f"{'='*40}")
    
    # Tokenization test
    inputs = tokenizer(text, return_tensors="pt")
    
    print("\nTokenization:")
    print(f"Input text: {text}")
    print(f"Token IDs: {inputs['input_ids'].tolist()[0]}")
    print(f"Tokens: {tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])}")
    
    # Forward pass test
    with torch.no_grad():
        outputs = model(**inputs)
    
    print("\nForward pass:")
    print(f"Logits shape: {outputs.logits.shape}")  # Should be [1, seq_len, vocab_size]
    
    # Embedding test
    input_embeds = model.get_input_embeddings()(inputs.input_ids)
    print("\nEmbeddings:")
    print(f"Embedding shape: {input_embeds.shape}")
    print(f"Mean embedding norm: {torch.norm(input_embeds, dim=2).mean().item():.4f}")
    
    # Test generation
    print("\nTest generation:")
    with torch.no_grad():
        generated = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=20,
            do_sample=True,
            top_k=50,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id
        )
        decoded = tokenizer.decode(generated[0], skip_special_tokens=False)
        print(f"Generated text: {decoded}")
    
    print(f"\n{'='*40}\n")


Testing Yakut text

Tokenization:
Input text: <|begin_of_text|>Араас сылларга үөрэнэн ааспыт училещаларын үбүлүөйүн көрсө кэллилэр ыраахтан-чугастан элбэх ыалдьыт
Token IDs: [0, 7931, 5567, 16944, 16780, 17172, 5546, 4560, 49601, 8374, 38655, 12512, 7897, 17392, 5281, 14709, 24109, 5366, 21, 27231, 32396, 7412, 49479]
Tokens: ['<|begin_of_text|>', 'Ар', '▁аас', '▁ сылларга ', '▁үөрэнэн ', '▁ааспыт ', '▁уч', '▁ил', '▁ещ', '▁аларын ', '▁үбүлүөй', '▁үн к', '▁өрс', '▁ө к', '▁элл', '▁илэр ', '▁ыраах', '▁тан', '-', 'чуг', '▁астан ', '▁элбэх ', '▁ыалдьыт']

Forward pass:
Logits shape: torch.Size([1, 23, 50000])

Embeddings:
Embedding shape: torch.Size([1, 23, 2048])
Mean embedding norm: 0.4007

Test generation:
Generated text: <|begin_of_text|>Ар аас  сылларга  үөрэнэн  ааспыт  уч ил ещ аларын  үбүлүөй үн к өрс ө к элл илэр  ыраах тан-чуг астан  элбэх  ыалдьыт-Ɵ-Ɵ-Ɵ-Ɵ-Ɵ-Ɵ-�



Testing Russian text

Tokenization:
Input text: <|begin_of_text|>Жил-был царь. Добрый царь, который очень любил свои

In [1]:
from transformers import AutoTokenizer
import os

# Load your custom tokenizer
tokenizer_path = "yakut-llama-model"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

def analyze_tokenization(text, tokenizer):
    # Encode the text
    inputs = tokenizer(text, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    # Calculate metrics
    char_count = len(text)
    token_count = len(tokens)
    compression_ratio = char_count / token_count
    
    # Reconstruct text from tokens
    reconstructed = tokenizer.decode(inputs['input_ids'][0])
    
    return {
        "original": text,
        "tokens": tokens,
        "token_count": token_count,
        "char_count": char_count,
        "compression_ratio": compression_ratio,
        "reconstructed": reconstructed,
        "perfect_reconstruction": text == reconstructed
    }

def print_tokenization_report(report):
    print(f"\nOriginal text ({report['char_count']} chars):")
    print(report['original'])
    
    print(f"\nTokens ({report['token_count']} tokens, ratio: {report['compression_ratio']:.2f} chars/token):")
    for i, token in enumerate(report['tokens']):
        print(f"{i:3d}: {token}")
    
    print("\nReconstructed text:")
    print(report['reconstructed'])
    print(f"Perfect reconstruction: {report['perfect_reconstruction']}")
    
    # Highlight problematic tokens
    problem_tokens = [token for token in report['tokens'] if len(token) == 1 or '�' in token]
    if problem_tokens:
        print("\nWarning: Problematic tokens detected:")
        print(problem_tokens)

# Test texts
test_texts = {
    "Yakut": "<|begin_of_text|>Араас сылларга үөрэнэн ааспыт училещаларын үбүлүөйүн көрсө кэллилэр ыраахтан-чугастан элбэх ыалдьыт",
    "Russian": "<|begin_of_text|>Жил-был царь. Добрый царь, который очень любил своих подданных и был с ними ласков.",
    "English": "<|begin_of_text|>Once upon a time there was a king. A kind king who loved his subjects dearly."
}

# Run analysis
for lang, text in test_texts.items():
    print(f"\n{'='*60}")
    print(f"TOKENIZATION ANALYSIS: {lang.upper()}")
    print(f"{'='*60}")
    
    report = analyze_tokenization(text, tokenizer)
    print_tokenization_report(report)


TOKENIZATION ANALYSIS: YAKUT

Original text (116 chars):
<|begin_of_text|>Араас сылларга үөрэнэн ааспыт училещаларын үбүлүөйүн көрсө кэллилэр ыраахтан-чугастан элбэх ыалдьыт

Tokens (23 tokens, ratio: 5.04 chars/token):
  0: <|begin_of_text|>
  1: Ар
  2: ▁аас
  3: ▁ сылларга 
  4: ▁үөрэнэн 
  5: ▁ааспыт 
  6: ▁уч
  7: ▁ил
  8: ▁ещ
  9: ▁аларын 
 10: ▁үбүлүөй
 11: ▁үн к
 12: ▁өрс
 13: ▁ө к
 14: ▁элл
 15: ▁илэр 
 16: ▁ыраах
 17: ▁тан
 18: -
 19: чуг
 20: ▁астан 
 21: ▁элбэх 
 22: ▁ыалдьыт

Reconstructed text:
<|begin_of_text|>Ар аас  сылларга  үөрэнэн  ааспыт  уч ил ещ аларын  үбүлүөй үн к өрс ө к элл илэр  ыраах тан-чуг астан  элбэх  ыалдьыт
Perfect reconstruction: False

['-']

TOKENIZATION ANALYSIS: RUSSIAN

Original text (100 chars):
<|begin_of_text|>Жил-был царь. Добрый царь, который очень любил своих подданных и был с ними ласков.

Tokens (40 tokens, ratio: 2.50 chars/token):
  0: <|begin_of_text|>
  1: Ж
  2: ▁ил
  3: -
  4: б
  5: ▁ыл 
  6: ▁ц
  7: ▁арь
  8: .
  9:  
 10: Д
 11